In [1]:
try:
    from lets_plot import *
except:
    print('Installing Lets plot...')
    !pip install lets_plot --quiet
    print('Installation done.')

Installing Lets plot...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 15.5 MB/s eta 0:00:00
Installation done.


In [5]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.7 MB/s eta 0:00:00


In [3]:
pip install colorama


In [7]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os

# Set your desired working directory path on Google Drive
desired_dir = '/content/drive/MyDrive/Spaceship_Titanic/spaceship-titanic'


# Change the working directory to the desired directory
os.chdir(desired_dir)

# Verify that the working directory has been changed
print(os.getcwd())


/content/drive/MyDrive/Spaceship_Titanic/spaceship-titanic


In [12]:
import sys
import pandas as pd
import numpy as np
from colorama import Fore, Back, Style
from lets_plot import *
LetsPlot.setup_html()

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,f1_score

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Data Prepration for EDA
Before starting the Exploratory Data Analysis (EDA), we need to perform some preparation steps:

**Break PassengerId** and **Cabin** into new features.

**Map** categorical variables to numbers.

Convert **CryoSleep** and **VIP** to booleans.

Drop the **Name** column since it does not contribute to the predictions.

All of these steps are stored in a variable, making it convenient to apply the same transformations to the test set later on.

In [13]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [14]:
# Train
print("\033[1;32mTrain Info\033[0m")
print("\033[;32mTrain Shape:\033[0m",df_train.shape)
print("\033[;32mTrain Missing Count:\033[0m")
print(df_train.isna().sum())
print('\n')

#Test
print("\033[1;32mTest Info\033[0m")
print("\033[;32mTest Shape:\033[0m",df_test.shape)
print("\033[;32mTest Missing Count:\033[0m")
print(df_test.isna().sum())


Train Info
Train Shape: (8693, 14)
Train Missing Count:
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


Test Info
Test Shape: (4277, 13)
Test Missing Count:
PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64


In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [16]:
# Preprocessing Functions
def split_PassengerId(df):
    Group_GroupNum = df.PassengerId.str.split('_', expand=True)
    Group_GroupNum = Group_GroupNum.astype(int)
    df[['PassengerGroup', 'PassengerGroupNum']] = Group_GroupNum
    return df.drop(columns='PassengerId')

def split_Cabin(df):
    Deck_Num_Side = df.Cabin.str.split('/', expand=True)
    df[['CabinDeck', 'CabinNum', 'CabinSide']] = Deck_Num_Side
    df.CabinNum = df.CabinNum.astype(float)
    return df.drop(columns='Cabin')

def cat2number(df, categories=None):
    cat_cols = ['CabinDeck', 'CabinSide', 'HomePlanet', 'Destination']
    create_categories = True if categories is None else False

    if create_categories:
        categories = {}

    for col in cat_cols:
        if create_categories:
            categories_ = df[col].value_counts().index.sort_values().to_list()
            categories[col] = categories_
        else:
            categories_ = categories[col]

        mapping = {key: i for i, key in enumerate(categories_)}
        df[col] = df[col].map(mapping)
        print(f'Mapped {col} {mapping}')

    return df

def add_age_groups(df):
    df['Age_group'] = np.nan
    conditions = [
        (df['Age'] <= 12),
        ((df['Age'] > 12) & (df['Age'] < 18)),
        ((df['Age'] >= 18) & (df['Age'] <= 25)),
        ((df['Age'] > 25) & (df['Age'] <= 30)),
        ((df['Age'] > 30) & (df['Age'] <= 50)),
        (df['Age'] > 50)
    ]
    age_groups = [
        'Age_0-12',
        'Age_13-17',
        'Age_18-25',
        'Age_26-30',
        'Age_31-50',
        'Age_51+'
    ]
    df['Age_group'] = np.select(conditions, age_groups, default=np.nan)
    return df

def bool2num(df):
    features = ['CryoSleep', 'VIP']
    df[features] = df[features].astype(bool)
    return df

def drop_columns(df, drop_columns=['Name']):
    return df.drop(columns=drop_columns)

def move_to_last(df):
    target = 'Transported'
    columns = df.columns.to_list()

    if target in columns:
        columns.remove(target)
        columns.append(target)

    return df[columns]

def prepare_dataset(df, funcs):
    df_ = df.copy()
    for func in funcs:
        df_ = func(df_)
    return df_

preproc_funcs = [split_PassengerId, split_Cabin, cat2number, add_age_groups, bool2num, drop_columns, move_to_last]

In [17]:
print("\033[1;32mPreparing Training Dataset\033[0m")
train = prepare_dataset(df_train, preproc_funcs)

print("\033[1;32mPreparing Testing Dataset\033[0m")
test = prepare_dataset(df_test, preproc_funcs)

Preparing Training Dataset
Mapped CabinDeck {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'T': 7}
Mapped CabinSide {'P': 0, 'S': 1}
Mapped HomePlanet {'Earth': 0, 'Europa': 1, 'Mars': 2}
Mapped Destination {'55 Cancri e': 0, 'PSO J318.5-22': 1, 'TRAPPIST-1e': 2}
Preparing Testing Dataset
Mapped CabinDeck {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'T': 7}
Mapped CabinSide {'P': 0, 'S': 1}
Mapped HomePlanet {'Earth': 0, 'Europa': 1, 'Mars': 2}
Mapped Destination {'55 Cancri e': 0, 'PSO J318.5-22': 1, 'TRAPPIST-1e': 2}


# Exploratory Data Analysis

In [18]:
train.shape

(8693, 17)

In [19]:
train.isna().sum().sort_values(ascending = False)

ShoppingMall         208
HomePlanet           201
CabinSide            199
CabinNum             199
CabinDeck            199
VRDeck               188
Spa                  183
FoodCourt            183
Destination          182
RoomService          181
Age                  179
CryoSleep              0
PassengerGroup         0
PassengerGroupNum      0
VIP                    0
Age_group              0
Transported            0
dtype: int64

In [20]:
train.describe()

,HomePlanet,Destination,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerGroup,PassengerGroupNum,CabinDeck,CabinNum,CabinSide
count,8492.000000,8511.000000,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8693.000000,8693.000000,8494.000000,8494.000000,8494.000000
mean,0.665214,1.483492,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791,4633.389624,1.517773,4.305392,600.367671,0.504827
std,0.798155,0.820237,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189,2671.028856,1.054241,1.778233,511.867226,0.500006
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2319.000000,1.000000,3.000000,167.250000,0.000000
50%,0.000000,2.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4630.000000,1.000000,5.000000,427.000000,1.000000
75%,1.000000,2.000000,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000,6883.000000,2.000000,6.000000,999.000000,1.000000
max,2.000000,2.000000,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,9280.000000,8.000000,7.000000,1894.000000,1.000000


More than half of the features require some imputation technique for missing values

# Correlation Matrix

In [21]:
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerGroup,PassengerGroupNum,CabinDeck,CabinNum,CabinSide,Age_group,Transported
0,1.0,False,2.0,39.0,False,0.0,0.0,0.0,0.0,0.0,1,1,1.0,0.0,0.0,Age_31-50,False
1,0.0,False,2.0,24.0,False,109.0,9.0,25.0,549.0,44.0,2,1,5.0,0.0,1.0,Age_18-25,True
2,1.0,False,2.0,58.0,True,43.0,3576.0,0.0,6715.0,49.0,3,1,0.0,0.0,1.0,Age_51+,False
3,1.0,False,2.0,33.0,False,0.0,1283.0,371.0,3329.0,193.0,3,2,0.0,0.0,1.0,Age_31-50,False
4,0.0,False,2.0,16.0,False,303.0,70.0,151.0,565.0,2.0,4,1,5.0,1.0,1.0,Age_13-17,True


In [22]:
corr = train.iloc[:,:-2].corr()
corr.style.background_gradient(cmap='RdYlGn', vmin=-0.6, vmax=0.6).set_sticky(axis=0)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerGroup,PassengerGroupNum,CabinDeck,CabinNum,CabinSide
HomePlanet,1.000000,0.085258,0.038559,0.138058,0.086213,0.215963,0.070782,0.103070,0.055232,0.040292,-0.006047,0.038778,-0.418904,-0.203172,0.000528
CryoSleep,0.085258,1.000000,-0.092895,-0.073771,-0.050342,-0.243434,-0.204765,-0.208465,-0.199075,-0.192749,-0.003709,0.062948,0.016852,-0.035881,0.016187
Destination,0.038559,-0.092895,1.000000,-0.007978,-0.035002,0.047685,-0.112180,0.025433,-0.058329,-0.073707,-0.003659,-0.033592,0.189937,0.105492,-0.011142
Age,0.138058,-0.073771,-0.007978,1.000000,0.058930,0.068723,0.130421,0.033133,0.123970,0.101007,-0.009439,-0.132756,-0.248003,-0.132255,0.011640
VIP,0.086213,-0.050342,-0.035002,0.058930,1.000000,0.050854,0.089639,-0.001104,0.037446,0.087322,0.016651,-0.002154,-0.118344,-0.059421,-0.002118
RoomService,0.215963,-0.243434,0.047685,0.068723,0.050854,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.000346,-0.022571,-0.026783,-0.008986,-0.012290
FoodCourt,0.070782,-0.204765,-0.112180,0.130421,0.089639,-0.015889,1.000000,-0.014228,0.221891,0.227995,-0.009552,0.023391,-0.324348,-0.183429,0.020149
ShoppingMall,0.103070,-0.208465,0.025433,0.033133,-0.001104,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.017397,-0.038812,-0.032926,0.001727,-0.020983
Spa,0.055232,-0.199075,-0.058329,0.123970,0.037446,0.010080,0.221891,0.013879,1.000000,0.153821,-0.005211,0.016820,-0.229111,-0.131342,0.002031
VRDeck,0.040292,-0.192749,-0.073707,0.101007,0.087322,-0.019581,0.227995,-0.007322,0.153821,1.000000,0.015432,0.010082,-0.257326,-0.139080,-0.008154


In [23]:
categorical_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
plots = []

for col in categorical_features:
    plot = ggplot(df_train) +\
           geom_bar(aes(x = col, fill = 'Transported', color = 'Transported'), alpha=0.4, size=1.5, position=position_dodge(width=0.6), width=0.5) +\
           ggtitle(col) +\
           theme(panel_grid_major_x='blank')
    plots.append(plot)

gggrid(plots, ncol = 2)

In [24]:
plot = ggplot(train) +\
       geom_bar(aes(x = 'Age_group', fill = 'Transported', color = 'Transported'), alpha=0.4, size=1.5, position=position_dodge(width=0.6), width=0.5) +\
           ggtitle('Age Group Distribution') +\
           theme(panel_grid_major_x='blank')

plot

# Distribution of Features

In [25]:
def plot_distributions(df):
    """
    Function to plot distribution plots of columns of a dataframe
    """
    columns = df.columns
    plots = []
    for col in columns:
        plot = ggplot(df) +\
               geom_histogram(aes(x=col, fill='Transported', color = 'Transported'), alpha = 0.4, size = 1.5) +\
               ggtitle(f'{col} Distribution') +\
               theme(panel_grid_major_x='blank')
        plots.append(plot)
    return plots

plots = plot_distributions(train)

gggrid(plots, ncol=2)

Most passengers are young, falling within the age group of 18 to 25. However,
the population of babies aged 0 to 2 is relatively high.

The majority of passengers opted not to use Cryosleep.

TRAPPIST-1e seems to be the preferred destination for most passengers.

There are only a few VIPs among the passengers.

Passenger groups are well-balanced in terms of their numbers.

The majority of groups consist of only one passenger.

The last CabinDecks are more densely populated compared to others.

Cabin numbers are concentrated in the lower range.

The distribution of amenities services is skewed due to a few very high values.





In [26]:
X_train, X_val, y_train, y_val = train_test_split(train.iloc[:,:-1], train.iloc[:,-1], test_size=0.25, random_state=1)

print("X Train Shape: ", X_train.shape)
print("X Valid Shape: ", X_val.shape)

X Train Shape:  (6519, 16)
X Valid Shape:  (2174, 16)


In [27]:
numeric_features = X_train.select_dtypes(exclude='object').columns
categorical_features = X_train.select_dtypes(include='object').columns

In [28]:
# Define Preprocessing Pipelines
numeric_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaling', StandardScaler())
])

categorical_preprocessor = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='constant'))
])

preprocessor = ColumnTransformer(transformers=[
    ('numeric', numeric_preprocessor, numeric_features),
    ('categorical', categorical_preprocessor, categorical_features)
])

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingClassifier())
])

model_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'PassengerGroup',
       'PassengerGroupNum', 'CabinDeck', 'CabinNum', 'CabinSide'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='constant'))]),
                                                  Index(['Age_group'], dtype='object'))])),
                ('model', GradientBoostingClassifier())])

In [29]:
def evaluate_pipeline_classification(pipe, model_name, X_train = X_train, y_train = y_train, X_test = X_val, y_test = y_val, random_state=42):

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    # Calculate classification scores
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    classification_scores = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    print(f"\033[1;32mModel: {model_name}\033[0m")
    print("\033[;32mAccuracy:\033[0m", classification_scores['accuracy'])
    print("\033[;32mPrecision:\033[0m", classification_scores['precision'])
    print("\033[;32mRecall:\033[0m", classification_scores['recall'])
    print("\033[;32mF1 Score:\033[0m", classification_scores['f1_score'])

In [30]:
evaluate_pipeline_classification(model_pipeline, 'Gradient Boosting Classifier')

Model: Gradient Boosting Classifier
Accuracy: 0.8150873965041399
Precision: 0.8082901554404145
Recall: 0.8387096774193549
F1 Score: 0.8232189973614776


In [31]:
model_pipeline2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBClassifier())
])

evaluate_pipeline_classification(model_pipeline2, 'XGBoost Classifier')

Model: XGBoost Classifier
Accuracy: 0.8068077276908924
Precision: 0.8186813186813187
Recall: 0.8010752688172043
F1 Score: 0.8097826086956522


In [32]:
params = {
    'activation': 'tanh',
    'alpha': 0.0001,
    'hidden_layer_sizes': (20,),
    'learning_rate': 'constant',
    'solver': 'adam'
}

model_pipeline3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', MLPClassifier(**params))
])

evaluate_pipeline_classification(model_pipeline3, 'MLP Classifier')

Model: MLP Classifier
Accuracy: 0.8141674333026679
Precision: 0.8201438848920863
Recall: 0.8172043010752689
F1 Score: 0.8186714542190305


In [33]:
ensemble_classifier = VotingClassifier(
    estimators=[
        ('Random Forest', RandomForestClassifier()),
        ('Gradient Boosting', GradientBoostingClassifier()),
        ('Support Vector Machine', SVC()),
        ('Ada Boosting', AdaBoostClassifier()),
        ('MLPClassifier', MLPClassifier(**params))
    ],
    voting='hard'
)

ensemble_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', ensemble_classifier)
])

ensemble_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'PassengerGroup',
       'PassengerGroupNum', 'CabinDeck', 'CabinNum', 'CabinSide'],
      dtype='object...
                                                                   SimpleImputer(strategy='constant'))]),
                                                  Index(['Age_group'], dtype='object'))])),
                ('model',
                 VotingClassifier(estimators=[('Random Forest',
                                               RandomForestClassifier()),
                                              ('Gradient Boosting',
                                               GradientBoostingClassifier()),
                                              ('Support Vector Machine', SVC()),
                                              ('Ada Boosting',
                                               AdaBoostClassifier()),
                                              ('MLPClassifier',
                                               MLPClassifier(activation='tanh',
                                                             hidden_layer_sizes=(20,)))]))])

In [34]:
evaluate_pipeline_classification(ensemble_pipeline, 'Ensemble Model')

Model: Ensemble Model
Accuracy: 0.81278748850046
Precision: 0.8085291557876414
Recall: 0.8324372759856631
F1 Score: 0.8203090507726271


# Final Submission

In [35]:
sample_submissions = pd.read_csv('sample_submission.csv')

final_submissions = ensemble_pipeline.predict(test)
sample_submissions['Transported'] = final_submissions
sample_submissions.to_csv('submission.csv', index = False)